# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
import json

In [2]:
import requests


In [3]:
# Your code here

def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("c:/Users/Owner/.secret/yelp_api.json")
api_key = keys['api_key']

term = 'mexican'
location = 'Atlanta, GA'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {'Authorization': 'Bearer {}'.format(api_key),}

url_params = {
                'term':term.replace(' ','+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
}

response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "kGb2Rnh3Uwni5k0qQY-05Q", "alias": "nuevo-laredo-cantina-atlanta", "name": "Nuevo Laredo Cantina", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/yd1--trCtw4RH3d6udVJ0w/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/nuevo-laredo-cantina-atlanta?adjust_creative=O3YSotnOLRMPePREWpdTng&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=O3YSotnOLRMPePREWpdTng", "review_count": 982, "categories": [{"alias": "mexican", "title": "Mexican"}], "rating": 4.0, "coordinates": {"latitude": 33.8094340325727, "longitude": -84.4367971831875}, "transactions": ["delivery", "pickup"], "price": "$$", "location": {"address1": "1495 Chattahoochee Ave NW", "address2": "", "address3": "", "city": "Atlanta", "zip_code": "30318", "country": "US", "state": "GA", "display_address": ["1495 Chattahoochee Ave NW", "Atlanta, GA 30318"]}, "phone": "+14043529009", "display_phone": "(404) 352-9009", "distance": 5851.36060579881

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [4]:
# Your code here; use a function or loop to retrieve all the results from your original request
for key in response.json().keys():
    print(key)
    value = response.json()[key]
    print(type(value))
    print('\n')
    print(len(response.json()['businesses']))
    
    

businesses
<class 'list'>


10
total
<class 'int'>


10
region
<class 'dict'>


10


In [5]:
print(response.json()['region'])

{'center': {'longitude': -84.37362670898438, 'latitude': 33.81321311652279}}


In [6]:
import pandas as pd
df = pd.DataFrame.from_dict(response.json()['businesses'])
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,nuevo-laredo-cantina-atlanta,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 33.8094340325727, 'longitude': -8...",(404) 352-9009,5851.360606,kGb2Rnh3Uwni5k0qQY-05Q,https://s3-media4.fl.yelpcdn.com/bphoto/yd1--t...,False,"{'address1': '1495 Chattahoochee Ave NW', 'add...",Nuevo Laredo Cantina,+14043529009,$$,4.0,982,"[delivery, pickup]",https://www.yelp.com/biz/nuevo-laredo-cantina-...
1,la-pastorcita-atlanta,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 33.84308, 'longitude': -84.32924}",(678) 705-8162,5278.910299,V86AJK3xytBv2C0bwi9_Ow,https://s3-media2.fl.yelpcdn.com/bphoto/SAFmOf...,False,"{'address1': '3304 Buford Hwy NE', 'address2':...",La Pastorcita,+16787058162,$,4.0,352,"[delivery, pickup]",https://www.yelp.com/biz/la-pastorcita-atlanta...
2,agavero-cantina-atlanta-atlanta,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 33.7891701465501, 'longitude': -8...",(404) 464-5250,5513.546998,m9OfK-nCIxMB-DCu0lU8Vg,https://s3-media1.fl.yelpcdn.com/bphoto/Ln4jMm...,False,"{'address1': '1133 Huff Rd NW', 'address2': 'S...",Agavero Cantina - Atlanta,+14044645250,$$,4.5,202,"[delivery, pickup]",https://www.yelp.com/biz/agavero-cantina-atlan...
3,the-original-el-taco-atlanta,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 33.786997, 'longitude': -84.355746}",(404) 873-4656,3350.751580,xI56hTOOHsQfk-DZviOrxQ,https://s3-media3.fl.yelpcdn.com/bphoto/cNunlM...,False,"{'address1': '1186 N Highland Ave NE', 'addres...",The Original El Taco,+14048734656,$$,4.0,421,[delivery],https://www.yelp.com/biz/the-original-el-taco-...
4,bone-garden-cantina-atlanta,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 33.7939834594727, 'longitude': -8...",(404) 418-9072,5296.725590,87O_yo8r8IlG7e82-usSiw,https://s3-media4.fl.yelpcdn.com/bphoto/HCa9GF...,False,"{'address1': '1425 Ellsworth Industrial Blvd',...",Bone Garden Cantina,+14044189072,$$,4.0,783,[],https://www.yelp.com/biz/bone-garden-cantina-a...


In [7]:
df.coordinates[1]



{'latitude': 33.84308, 'longitude': -84.32924}

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [8]:
# Your code here
response.json()['region']
import folium

In [9]:
long = -84.37362670898438 
lat = 33.81321311652279

base_map = folium.Map([lat, long], zoom_start=13)
base_map

    
    

In [14]:
for row in df.index:
    try:
        lat_long = df['coordinates'][row]
        lat = lat_long['latitude']
        long = lat_long['longitude']
        name = df['name'][row]
        popup = folium.Popup(name, parse_html=True)
        marker = folium.Marker([lat, long], popup=popup)
        marker.add_to(base_map)
    except:
        print(row + 'error')

base_map

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!